# Revista de Administração, Contabilidade e Economia – RACE

## Bibliotecas importadas

In [1]:
import requests
from bs4 import BeautifulSoup

import pandas as pd

## Funções básicas

In [2]:
def criar_query(key_word):
    return "+".join(key_word.split())

In [3]:
def listar_links_pagina(url_search):
    page = requests.get(url_search)
    soup = BeautifulSoup(page.content, 'html.parser')
    results = soup.find_all('h3', {'class': 'title'})
    a_links = []
    for item in results:
        try:
            remover_links = 'Editorial' in item.find('a').text
            remover_links = remover_links or 'Folha de rosto' in item.find('a').text
            remover_links = remover_links or 'Dados da edição ' in item.find('a').text
            if not remover_links:
                a_links.append(item.find('a')['href'])
        except:
            pass
    return a_links

In [4]:
def listar_links(url_search_text, query, pg_num=1):
    a_links = [] 
    while True:
        url_search = url_search_text.replace('__query__', query).replace('__pg_num__', str(pg_num))
        tmp = listar_links_pagina(url_search)
        if len(tmp) != 0:
            pg_num += 1
            a_links += tmp
        else:
            break
    return a_links

In [5]:
url_search_text = 'https://portalperiodicos.unoesc.edu.br/race/search/index?query=__query__&searchJournal=15&authors=&dateFromMonth=&dateFromDay=&dateFromYear=2017&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'
links = listar_links(url_search_text, 'contabilidade+ambiental', pg_num=1)

link = links[0]

link


'https://portalperiodicos.unoesc.edu.br/race/article/view/19715'

In [6]:
links

['https://portalperiodicos.unoesc.edu.br/race/article/view/19715',
 'https://portalperiodicos.unoesc.edu.br/race/article/view/27352',
 'https://portalperiodicos.unoesc.edu.br/race/article/view/12053',
 'https://portalperiodicos.unoesc.edu.br/race/article/view/24552',
 'https://portalperiodicos.unoesc.edu.br/race/article/view/17424',
 'https://portalperiodicos.unoesc.edu.br/race/article/view/10183',
 'https://portalperiodicos.unoesc.edu.br/race/article/view/23354',
 'https://portalperiodicos.unoesc.edu.br/race/article/view/21762',
 'https://portalperiodicos.unoesc.edu.br/race/article/view/13048',
 'https://portalperiodicos.unoesc.edu.br/race/article/view/12929']

In [7]:
page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

In [8]:
def completar_lista(lista):
    if len(lista) < 6:
        lista += (6 - len(lista)) * ['']
    return lista

In [9]:
def limpar_afiliacao(afiliacao):
    afiliacao = ' '.join(afiliacao.split())
    afiliacao = afiliacao.split('-')[0]
    afiliacao = afiliacao.split('(')[0]
    return afiliacao.strip()

In [10]:
def captura_titulo(soup):
    try:
        title = soup.find('h1', {'class': 'page_title'}).text
        title = ' '.join(title.split())
    except:
        title = ''
    return {'Título': title}

In [11]:
captura_titulo(soup)

{'Título': 'Adoção de “Gestão Estratégica” por uma empresa distribuidora de derivados de petróleo'}

In [12]:
def captura_data(soup):
    try:
        date_tag = soup.find('div', {'class': 'item published'})
        date = date_tag.find('div', {'class': 'value'}).text
        date = ' '.join(date.split())
    except:
        date = ''
    return {'Data de Publicação': date[:10]}

In [13]:
captura_data(soup)

{'Data de Publicação': '12/16/2019'}

In [14]:
def captura_resumo(soup):
    try:
        abstract_section = soup.find('section', {'class': 'item abstract'})
        abstract = abstract_section.text
        abstract = abstract.replace('Resumo:', '')
        abstract = abstract.replace('Resumo', '').strip()
        #abstract = ' '.join(abstract.split())
    except:
        abstract = ''
    return {'Resumo': abstract}

In [15]:
captura_resumo(soup)

{'Resumo': 'A perenidade das organizações somente é possível com a adoção de ferramentas e técnicas de administração adequadas, capazes de responder rapidamente às mudanças e necessidades do mercado. Em razão disso, teve-se o objetivo de explorar as características da gestão de uma empresa distribuidora de derivados de petróleo com vistas à adoção de uma “gestão estratégica”, sob o método relato técnico-científico. Durante o estudo, constatou-se que a gestão realizada por alguns dos socioproprietários fundadores não conta com ferramentas e técnicas que permitam que sucessores ou outros colaboradores possam atuar sem a participação destes e inovem frente às mudanças do mercado. Diante das constatações, duas intervenções foram propostas: a adoção de um modelo de desempenho organizacional com uso de diversos indicadores de gestão, e o uso da ferramenta Canvas, a qual permite descrever, analisar e projetar modelos de negócios relacionados ou não com algum tipo de estratégia ou gerenciament

In [16]:
def captura_palavras_chave(soup):
    try:
        kw_section = soup.find('section', {'class': 'item keywords'})
        kw = kw_section.text.replace('Palavras-chave:', '')
        kw = ' '.join(kw.split())
        kw = kw.replace(',', '.').replace(';', '.')
    except:
        kw = ''
    if kw == '':
        try:
            kw_section = soup.find('section', {'class': 'item abstract'})
            p_list = kw_section.find_all('p')
            for p in p_list:
                if 'Palavras-chave' in p.text:
                    kw_full = p.text
                    kw = kw_full.replace('Palavras-chave', '').replace(':', '')
                    kw = ' '.join(kw.split())
                    kw = kw.replace(',', '.').replace(';', '.')
        except:
            pass
    return {'Palavras-chave': kw}

In [17]:
captura_palavras_chave(soup)

{'Palavras-chave': 'Avaliação de desempenho. Gestão estratégica. Intervenção organizacional. Profissionalização da gestão'}

In [18]:
def captura_autores(soup):
    autores = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        autores = authors_ul.find_all('span', {'class': 'name'})
        autores = [' '.join(a.text.split()) for a in autores]
    except:
        pass
    qtd_autores = len(autores)
    autores = completar_lista(autores)
    autores = [qtd_autores] + autores
    
    key_autores = [f"Autor {i}" for i in range(1, 7)]
    key_autores = ['Qtd. de Autores'] + key_autores

    return dict(zip(key_autores, autores))

In [19]:
captura_autores(soup)

{'Qtd. de Autores': 2,
 'Autor 1': 'Luciano Maldonado Felipe',
 'Autor 2': 'Claudio Antonio Rojo',
 'Autor 3': '',
 'Autor 4': '',
 'Autor 5': '',
 'Autor 6': ''}

In [20]:
def captura_afiliacao(soup):
    afiliacao = []
    try:
        authors_ul = soup.find('ul', {'class': 'authors'})
        afiliacao = authors_ul.find_all('span', {'class': 'affiliation'})
        afiliacao = [' '.join(a.text.split()) for a in afiliacao]
        afiliacao = [limpar_afiliacao(a) for a in afiliacao]
    except:
        pass
    afiliacao = completar_lista(afiliacao)
    
    key_affiliation = [f"Afiliação {i}" for i in range(1, 7)]

    return dict(zip(key_affiliation, afiliacao))

In [21]:
captura_afiliacao(soup)

{'Afiliação 1': 'UNIOESTE',
 'Afiliação 2': 'UNIOESTE',
 'Afiliação 3': '',
 'Afiliação 4': '',
 'Afiliação 5': '',
 'Afiliação 6': ''}

In [22]:
def realizar_consulta(key_words_list):
    periodico_nome_dict = {'Revista': 'Revista de Administração, Contabilidade e Economia – RACE'}
    url_search_text = 'https://portalperiodicos.unoesc.edu.br/race/search/index?query=__query__&searchJournal=15&authors=&dateFromMonth=&dateFromDay=&dateFromYear=2012&dateToMonth=&dateToDay=&dateToYear=2021&searchPage=__pg_num__#results'
    a_links = []
    for key_word in key_words_list:
        query = criar_query(key_word)
        a_links += listar_links(url_search_text, query)
        
    resultados = []
    for link in a_links:
        link_dict = {'Url Artigo': link}
        page = requests.get(link)
        soup = BeautifulSoup(page.content, 'html.parser')
        dict_artigo = {
            **captura_data(soup),
            **captura_titulo(soup),
            **captura_resumo(soup),
            **captura_palavras_chave(soup),
            **captura_autores(soup),
            **captura_afiliacao(soup),
            **link_dict,
            **periodico_nome_dict
        }
        resultados.append(dict_artigo)
    df = pd.DataFrame(resultados)
    df['Data de Publicação'] = pd.to_datetime(df['Data de Publicação'], format='%m/%d/%Y')
    return df

## Pesquisa por artigos

In [23]:
key_words_list = ['contabilidade ambiental', 'balanço social', 'relato integrado', 'nbct 15']
df = realizar_consulta(key_words_list)
df.head(3)

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2013-10-14,PERFIL DOS ARTIGOS QUE UTILIZARAM A ESTRATÉGIA...,Este estudo objetiva identificar o perfil dos ...,,2,Ilse Maria Beuren,Rosemar José Hall,,,,,Universidade Federal do Paraná,Universidade Regional de Blumenau,,,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."
1,2014-01-29,EVIDENCIAÇÃO AMBIENTAL VOLUNTÁRIA DE COMPANHIA...,O estudo objetivou analisar quais informações ...,,4,Francisca Francivânia Rodrigues Ribeiro Macêdo,Geovanne Dias de Moura,Vanderlei Gollo,Roberto Carlos Klann,,,Universidade Regional de Blumenau,Universidade Regional de Blumenau,Universidade Regional de Blumenau,Universidade Regional de Blumenau,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."
2,2021-08-31,Relação entre desempenho sustentável e desempe...,A sociedade têm demonstrado relevante atenção ...,Desempenho sustentável. Desempenho de mercado....,3,Janine Patrícia Jost,Adriana Kroenke,Nelson Hein,,,,FURB,FURB,FURB,,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."


In [24]:
df.to_csv('../data/RACE.csv', index=False)

In [25]:
df

,Data de Publicação,Título,Resumo,Palavras-chave,Qtd. de Autores,Autor 1,Autor 2,Autor 3,Autor 4,Autor 5,Autor 6,Afiliação 1,Afiliação 2,Afiliação 3,Afiliação 4,Afiliação 5,Afiliação 6,Url Artigo,Revista
0,2013-10-14,PERFIL DOS ARTIGOS QUE UTILIZARAM A ESTRATÉGIA...,Este estudo objetiva identificar o perfil dos ...,,2,Ilse Maria Beuren,Rosemar José Hall,,,,,Universidade Federal do Paraná,Universidade Regional de Blumenau,,,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."
1,2014-01-29,EVIDENCIAÇÃO AMBIENTAL VOLUNTÁRIA DE COMPANHIA...,O estudo objetivou analisar quais informações ...,,4,Francisca Francivânia Rodrigues Ribeiro Macêdo,Geovanne Dias de Moura,Vanderlei Gollo,Roberto Carlos Klann,,,Universidade Regional de Blumenau,Universidade Regional de Blumenau,Universidade Regional de Blumenau,Universidade Regional de Blumenau,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."
2,2021-08-31,Relação entre desempenho sustentável e desempe...,A sociedade têm demonstrado relevante atenção ...,Desempenho sustentável. Desempenho de mercado....,3,Janine Patrícia Jost,Adriana Kroenke,Nelson Hein,,,,FURB,FURB,FURB,,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."
3,2014-08-25,DEZOITO ANOS DE PRODUÇÃO ACADÊMICA DO CAMPO DO...,O objetivo deste estudo foi analisar o perfil ...,Contabilidade. Produção acadêmica. Periódicos ...,1,Henrique César Melo Ribeiro,,,,,,Prof. do Mestrado Profissional em Administraçã...,,,,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."
4,2014-01-29,REDES DE LOGÍSTICA REVERSA: UM ESTUDO DO CANAL...,Na indústria do plástico a logística reversa o...,,4,Fernando Antonio Tenório,Audálio Fernandes dos Reis,Débora Eleonora Silva,Maria Conceição Melo Silva Luft,,,Universidade Federal de Sergipe,Universidade Federal de Sergipe,Universidade Federal de Sergipe,Universidade Federal de Sergipe,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."
5,2014-02-17,O PROCESSO DE ADAPTAÇÃO ESTRATÉGICA DA ESCOLA ...,Este artigo é resultante de uma pesquisa a qua...,Adaptação Estratégica Organizacional. Mudança....,2,Mariana Pereira Querino,Mário César Barreto Moraes,,,,,Universidade do Estado de Santa Catarina,,,,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."
6,2014-03-12,ANÁLISE DA CONFORMIDADE DAS INFORMAÇÕES DIVULG...,O estudo objetivou analisar o grau de conformi...,Conformidade das informações divulgadas. Ativo...,2,Geovanne Dias de Moura,Patrícia Siqueira Varela,,,,,Universidade Regional de Blumenau,Universidade Federal de São Paulo,,,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."
7,2014-03-12,CARACTERÍSTICAS EXPLICATIVAS DO NÍVEL DE DISCL...,O objetivo desta pesquisa foi identificar as c...,Accountability. Municípios. Teoria da divulgaç...,4,Bruna Camargos Avelino,Jacqueline Veneroso Alves da Cunha,Gerlando Augusto Sampaio Franco de Lima,Romualdo Douglas Colauto,,,Universidade Federal de Minas Gerais,Universidade Federal de Minas Gerais,Universidade de São Paulo,Universidade Federal do Paraná,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."
8,2014-03-12,GOVERNANÇA CORPORATIVA EM EMPRESAS PÚBLICAS,Este estudo teve o objetivo de investigar como...,Governança Corporativa. Empresa pública. Groun...,4,Milton de Abreu Campanário,Sergio Nunes Muritiba,Patricia Morilha Muritiba,Henrique César Melo Ribeiro,,,"Universidade de São Paulo, Faculdade de Econom...",Faculdades Metropolitanas Unidas,Faculdades Metropolitanas Unidas,Faculdade Maurício de Nassau,,,https://portalperiodicos.unoesc.edu.br/race/ar...,"Revista de Administração, Contabilidade e Econ..."
9,2017-08-30,Is sustainability in business strategy factu